In [110]:
import pandas as pd
import re
from io import StringIO
import numpy as np
import base64

In [111]:
df = pd.read_csv('labelled_enron.csv')
df

,column_a,message_id,date,from,to,subject,x_from,x_to,x_cc,x_bcc,...,cat_10_level_1,cat_10_level_2,cat_10_weight,cat_11_level_1,cat_11_level_2,cat_11_weight,cat_12_level_1,cat_12_level_2,cat_12_weight,labeled
0,379,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15T14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,381,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15T14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2139,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20T17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2140,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20T17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2232,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09T12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,499650,<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05T13:42:45,frozenset({'louise.kitchen@enron.com'}),"frozenset({'greg.whalley@enron.com', 'john.lav...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1698,501042,<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13T14:00:00,frozenset({'steven.kean@enron.com'}),"frozenset({'greg.whalley@enron.com', 'kenneth....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1699,507376,<205897.1075861997314.JavaMail.evans@thyme>,2001-11-13T22:20:01,frozenset({'jeanie.slone@enron.com'}),frozenset({'mark.whitt@enron.com'}),RE: confidential ee info,"Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1700,509660,<23743848.1075863311776.JavaMail.evans@thyme>,2001-07-11T15:29:22,frozenset({'legalonline-compliance@enron.com'}),"frozenset({'williams@mailman.enron.com', 'bwil...",Confidential Information and Securities Trading,Office of the Chairman - Enron Wholesale Servi...,"WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [112]:
df=df[~df['content'].isna()]
df=df[df['labeled']==True]
col=['message_id','subject','content','cat_1_level_2']
df=df.loc[:,col]
df.head()

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,1.0
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,3.0
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",4.0
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",4.0
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",4.0


In [113]:
# Create a new 'label' column based on 'cat_1_level_2'
df['label'] = df['cat_1_level_2'].apply(lambda x: 1.0 + (x / 10.0))

# Map 'label' values to the desired labels
label_mapping = {
    1.1: 'Company Business/Strategy',
    1.2: 'Purely Personal',
    1.3: 'Personal but in professional context',
    1.4: 'Logistic Arrangements',
    1.5: 'Employment arrangements',
    1.6: 'Document editing/checking/collaboration',
    1.7: 'Empty message (due to missing attachment)',
    1.8: 'Empty message'
}

df['label'] = df['label'].map(label_mapping)
df.drop(columns=['cat_1_level_2'], inplace=True)
df

,message_id,subject,content,label
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,Company Business/Strategy
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,Personal but in professional context
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",Logistic Arrangements
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",Logistic Arrangements
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",Logistic Arrangements
...,...,...,...,...
1697,<18361957.1075861368310.JavaMail.evans@thyme>,Confidential Due Diligence Info - GS,There is a lot of confidential information her...,Company Business/Strategy
1698,<29049822.1075842045435.JavaMail.evans@thyme>,California's dim bulbs,You may have seen this when it first came out ...,Company Business/Strategy
1699,<205897.1075861997314.JavaMail.evans@thyme>,RE: confidential ee info,"Mark, Can you email me or fax me any written d...",Employment arrangements
1700,<23743848.1075863311776.JavaMail.evans@thyme>,Confidential Information and Securities Trading,"To:WILLIAMS, WILLIAM Email:bwillia5@enron.com ...",Company Business/Strategy


In [114]:
df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)
df.to_csv('simplified_labeled_enron.csv', index=False)
df.head()

,message_id,subject,body,category
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,Company Business/Strategy
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,Personal but in professional context
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",Logistic Arrangements
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",Logistic Arrangements
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",Logistic Arrangements


In [115]:
# Split into train and test dataset
original_df = pd.read_csv('simplified_labeled_enron.csv')

# Shuffle the rows randomly
# shuffled_df = original_df.sample(frac=1, random_state=42)  # Use a fixed random_state for reproducibility
shuffled_df = original_df.sample(frac=1)

# Split the DataFrame into two parts
train_df = shuffled_df.iloc[:1400]
test_df = shuffled_df.iloc[1400:]

# Save the two split DataFrames to separate CSV files
train_df.to_csv('simplified_labeled_enron_train.csv', index=False)
test_df.to_csv('simplified_labeled_enron_test.csv', index=False)

In [116]:
# function to create tuning prompts
def create_train_prompt(row):
    category = str(row['category'])  
    subject = str(row['subject'])    
    body = str(row['body'])          
    
    # categorization train prompt with placeholders for subject, body, and category
    categorization_train_prompt = (
        "<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
        "1. 'Company Business/Strategy'\n"
        "2. 'Purely Personal'\n"
        "3. 'Personal but in a professional context'\n"
        "4. 'Logistic Arrangements'\n"
        "5. 'Employment arrangements'\n"
        "6. 'Document editing/checking/collaboration'\n"
        "7. 'Empty message (due to missing attachment)'\n"
        "8. 'Empty message'\n"
        "Please provide only one category (e.g., 'Purely Personal'). <</SYS>>\n"
        "\n"
        "Subject:: {{subject}}\n"
        "Body:: {{body}}\n"
        "What should this email be categorized as?\n" 
        "[/INST] {{category}} <s>"
    )
    
    # replacing placeholders with actual values
    prompt = categorization_train_prompt.replace("{{subject}}", subject)
    prompt = prompt.replace("{{body}}", body)
    prompt = prompt.replace("{{category}}", category)
    
    return prompt

train_df['prompts'] = train_df.apply(create_train_prompt, axis=1)
train_df['prompts'][0]


/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_77263/3755537603.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['prompts'] = train_df.apply(create_train_prompt, axis=1)


"<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. 'Company Business/Strategy'\n2. 'Purely Personal'\n3. 'Personal but in a professional context'\n4. 'Logistic Arrangements'\n5. 'Employment arrangements'\n6. 'Document editing/checking/collaboration'\n7. 'Empty message (due to missing attachment)'\n8. 'Empty message'\nPlease provide only one category (e.g., 'Purely Personal'). <</SYS>>\n\nSubject:: Re: Confidential Employee Information/Lenhart\nBody:: I also need to know the base salaries of Jay Reitmeyer and Monique Sanchez. They are doing the same job as Matt.\nWhat should this email be categorized as?\n[/INST] Company Business/Strategy <s>"

In [117]:
# creating training dataset
prompts_df = pd.DataFrame(columns=['prompts'])
prompts_df['prompts'] = train_df['prompts'].astype(str)
prompts_df = prompts_df.drop(prompts_df[prompts_df['prompts'] == 'nan'].index)
prompts_df.reset_index(drop=True, inplace=True)
prompts_df.to_csv('prompts_train.csv', index=False)
prompts_df

,prompts
0,<s>[INST] <<SYS>> You are reviewing an email u...
1,<s>[INST] <<SYS>> You are reviewing an email u...
2,<s>[INST] <<SYS>> You are reviewing an email u...
3,<s>[INST] <<SYS>> You are reviewing an email u...
4,<s>[INST] <<SYS>> You are reviewing an email u...
...,...
1395,<s>[INST] <<SYS>> You are reviewing an email u...
1396,<s>[INST] <<SYS>> You are reviewing an email u...
1397,<s>[INST] <<SYS>> You are reviewing an email u...
1398,<s>[INST] <<SYS>> You are reviewing an email u...


In [118]:
prompts_df['prompts'][4]

"<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. 'Company Business/Strategy'\n2. 'Purely Personal'\n3. 'Personal but in a professional context'\n4. 'Logistic Arrangements'\n5. 'Employment arrangements'\n6. 'Document editing/checking/collaboration'\n7. 'Empty message (due to missing attachment)'\n8. 'Empty message'\nPlease provide only one category (e.g., 'Purely Personal'). <</SYS>>\n\nSubject:: In Phoenix, AZ to talk to Catholic Diocene Physical Plant Managers\n -\nBody:: Lay the groundwork and educate them about what's going on around the country from a regulatory standpoint, sensitize them to coming opportunities. Also, Steve Barth will be there to talk about creative products / services we can offer. 1:00 - 2:30 presentation, then q&a, then cocktails and dinner sponsored by Enron. Per Gary von Fischer 614-792-6010\nWhat should this email be cate

In [119]:
# function to create testing prompts
def create_test_prompt(row): 
    subject = str(row['subject'])    
    body = str(row['body'])        
    
    # categorization test prompt with placeholders for subject and body
    categorization_test_prompt = (
        "You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
        "1. 'Company Business/Strategy'\n"
        "2. 'Purely Personal'\n"
        "3. 'Personal but in a professional context'\n"
        "4. 'Logistic Arrangements'\n"
        "5. 'Employment arrangements'\n"
        "6. 'Document editing/checking/collaboration'\n"
        "7. 'Empty message (due to missing attachment)'\n"
        "8. 'Empty message'\n"
        "Please provide only one category (e.g., 'Purely Personal').\n"
        "\n"
        "Subject:: {{subject}}\n"
        "Body:: {{body}}\n"
        "What should this email be categorized as?\n"
    )
    
    # replacing placeholders with actual values
    prompt = categorization_test_prompt.replace("{{subject}}", subject)
    prompt = prompt.replace("{{body}}", body)
    
    return prompt

test_df['input_test_prompts'] = test_df.apply(create_test_prompt, axis=1)

# creating testing dataset
prompts_df_test = pd.DataFrame(columns=['input_test_prompts', 'category'])
prompts_df_test['input_test_prompts'] = test_df['input_test_prompts'].astype(str)
prompts_df_test['category'] = test_df['category'].astype(str)
prompts_df_test = prompts_df_test.drop(prompts_df_test[prompts_df_test['input_test_prompts'] == 'nan'].index)
prompts_df_test.reset_index(drop=True, inplace=True)
prompts_df_test.to_csv('prompts_test.csv', index=False)
prompts_df_test

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_77263/1221596927.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['input_test_prompts'] = test_df.apply(create_test_prompt, axis=1)


,input_test_prompts,category
0,You are reviewing an email using its subject a...,Company Business/Strategy
1,You are reviewing an email using its subject a...,Company Business/Strategy
2,You are reviewing an email using its subject a...,Logistic Arrangements
3,You are reviewing an email using its subject a...,Document editing/checking/collaboration
4,You are reviewing an email using its subject a...,Logistic Arrangements
...,...,...
292,You are reviewing an email using its subject a...,Personal but in professional context
293,You are reviewing an email using its subject a...,Document editing/checking/collaboration
294,You are reviewing an email using its subject a...,Logistic Arrangements
295,You are reviewing an email using its subject a...,Logistic Arrangements


Checking a few prompts...

In [120]:
prompts_df_test['input_test_prompts'][7]

'You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. \'Company Business/Strategy\'\n2. \'Purely Personal\'\n3. \'Personal but in a professional context\'\n4. \'Logistic Arrangements\'\n5. \'Employment arrangements\'\n6. \'Document editing/checking/collaboration\'\n7. \'Empty message (due to missing attachment)\'\n8. \'Empty message\'\nPlease provide only one category (e.g., \'Purely Personal\').\n\nSubject:: Re: White House Talking Points and Abraham Mtg with EPSA\nBody:: Linda - as we discussed, adding a sentence on open access is key. Also, I would delete the sentence about the transmission system not being designed for the way it\'s being used today. Richard Shapiro 04/02/2001 04:23 AM To: Linda Robertson/NA/Enron cc: Steven J Kean/NA/Enron@Enron, Mark Palmer/Corp/Enron@ENRON, Joe Hartsoe/Corp/Enron Subject: Re: White House Talking Points and Abraham Mtg with EPSA It 